In [5]:
path = '../tyc_cm/'

In [2]:
# %load_ext autoreload
# %autoreload 2
import dgl
import torch

Using backend: pytorch


### brand
### brand-organize: 融资历程
### brand-brand: 竞品信息

In [6]:
# >>> graph_data = {
# ...    ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
# ...    ('drug', 'interacts', 'gene'): (th.tensor([0, 1]), th.tensor([2, 3])),
# ...    ('drug', 'treats', 'disease'): (th.tensor([1]), th.tensor([2]))
# ... }

In [12]:
brands = set()
brand_belong = []
brand_rongzi = []
brand_jingpin = []
k = 1
for line in open(path + 'brand_all.json'):
    line = eval(line)
    if k:
        print(line)
        k = 0
    brand_id = line['id']
    brands.add(brand_id)
    brand_belong.append((brand_id, line['公司ID'], 'brand_belong'))
    brand_rongzi.extend([(brand_id, i['id'], 'brand_rongzi') for d in line.get('融资历程', []) for i in d['投资方']])
    brand_jingpin.extend([(brand_id, i, 'brand_jingpin') for i in line.get('竞品信息', [])])

{'地址': '江苏', '公司名称': '扬州市伏尔坎机械制造有限公司', '公司ID': 'company/2320405693', '名称': '伏尔坎机', '融资轮次': 'A轮', '成立日期': '2007-03-26', '标签': ['机械制造', '机械设备', '工业机械', '生产制造', '铁路运输设备', '铁路、船舶、航空航天和其他运输设备制造业'], '简介': '扬州市伏尔坎机械制造有限公司是专业定制多功能智能运输装备的设计制造企业，产品供应范围专注于航天、军工、船舶、能源、精密仪器、化工及重大工程和极限运输领域。\xa0\xa0', '核心成员': [{'姓名': '朱其安', 'ID': 'human/1972012247-c2320405693', '职位': '执行董事&联合创始人', '简介': '朱其安。扬州市伏尔坎机械制造有限公司联合创始人、法人代表、执行董事。'}, {'姓名': '张绍岩', 'ID': 'human/1931010842-c2320405693', '职位': '股东', '简介': '-'}], '融资历程': [{'披露日期': '2019-04-03', '交易金额': '未披露', '融资轮次': 'A轮', '估值': '-', '比例': '-', '投资方': [{'id': 'organize/baf302886', '投资方名称': '明势资本'}]}, {'披露日期': '2018-04-25', '交易金额': '未披露', '融资轮次': 'Pre-A轮', '估值': '-', '比例': '-', '投资方': [{'id': 'organize/bf78410875', '投资方名称': '友和资本'}]}], '竞品信息': ['brand/b1fd8150263', 'brand/bee1b172540', 'brand/bc991157174'], 'id': 'brand/b64db316846'}


In [4]:
brand_graph = [brand_belong, brand_rongzi, brand_jingpin]

In [14]:
len(brands), len(brand_belong), len(brand_rongzi), len(brand_jingpin)

(55056, 55056, 156921, 3379889)

In [8]:
brand_rongzi[0]

('brand/b64db316846', 'organize/baf302886', 'brand_rongzi')

### company
### 股东信息: company-human, company-company
### 对外投资: company-company
### 竞品信息: company-company
### 供应商: company-company
### 历史股东: company-human, company-company

In [12]:
comps = set()
comp_gudong = []
comp_touzi = []
comp_jingpin = []
comp_gongying = []
comp_lsgudong = []
k = 1

for line in open('../data/raw/company_total.json'):
    line = eval(line)
    if k:
        print(line)
        print(line.keys())
        k = 0
    comp_id = line['id']
    comps.add(comp_id)
    comp_gudong.extend([(comp_id, d.get('CompanyID') or d.get('HumanID') if isinstance(d, dict) else d, 'comp_gudong') for d in line.get('股东信息', [])])
    comp_touzi.extend([(comp_id, d.get('CompanyID') or d.get('HumanID') if isinstance(d, dict) else d, 'comp_touzi') for d in line.get('对外投资', [])])
    comp_jingpin.extend([(comp_id, d.get('CompanyID') if isinstance(d, dict) else d, 'comp_jingpin') for d in line.get('竞品信息', [])])
    comp_gongying.extend([(comp_id, d.get('CompanyID') if isinstance(d, dict) else d, 'comp_gongying') for d in line.get('供应商', [])])
    comp_lsgudong.extend([(comp_id, d.get('CompanyID') or d.get('HumanID'), 'comp_lsgudong') for d in line.get('历史股东', [])])

{'地址': '浙江省杭州市余杭区余杭街道文一西路1818-2号9幢806室', '简介': '仅20年，互联网就完成了“传统IT行业—互联网—移动互联网—人工智能—IOT”的更迭，技术直接对世界带来影响所用的时间从10年缩至5年，且速度越来越快。优课达团队希望通过累积的经验，为热爱编程的同学们规划科学的学习路径，更快更好地学写代码；通过各种测试让同学们对自己有更加清晰的认知； 通过简历指导和大厂内推，让每个优秀的人都有闪光的机会！', '名称': '优课达（杭州）网络有限公司', '股东信息': [{'股东名称': '焦英俊', 'HumanID': 'human/2058912128-c3320142871', '持股比例': '82.52%', '最终受益股份': '82.52%', '日期': '2037-08-30'}, {'股东名称': '上海阿米巴佰晖创业投资合伙企业（有限合伙）', 'CompanyID': 'company/26962466', '持股比例': '17.48%', '最终受益股份': '17.48%', '日期': '-'}], '企业业务': [{'产品名称': '优课达', 'BrandID': 'brand/b43c0469752', '所属地': '浙江', '标签': '教育培训', '日期': '2017-09-12'}], '竞品信息': [{'企业名称': '-', '日期': '2016-09-08', '产品标签': '-', '所属地': '-', '简介': '-'}, {'产品名称': '世拓电气', 'BrandID': 'brand/b4a76416531', '日期': '2014-05-07', '产品标签': '先进制造', '所属地': '江苏', '简介': '程序设计公司', '企业名称': '无锡世拓电气有限公司', 'CompanyID': 'company/2318666755'}, {'产品名称': '智汇美科技', 'BrandID': 'brand/be0e0393127', '日期': '-', '产品标签': '先进制造', '所属地': '广东', '简介': '编程服务商', '企业名称': '深圳市智汇美科技有限公司', 'CompanyID': 'company/30

In [13]:
comp_graph = [comp_gudong, comp_touzi, comp_jingpin, comp_gongying, comp_lsgudong]

In [14]:
len(comps), len(comp_gudong), len(comp_touzi), len(comp_jingpin), len(comp_gongying), len(comp_lsgudong)

(92524, 450083, 425959, 1483711, 187471, 161396)

In [32]:
len(set(list(zip(*brand_belong))[1]))

54996

In [36]:
bs, cs, _ = list(zip(*brand_belong))

In [37]:
dd = {k: [] for k in cs}

In [39]:
for b, c, _ in brand_belong:
    dd[c].append(b)

In [40]:
dd

{'company/2320405693': ['brand/b64db316846'],
 'company/115538760': ['brand/bfaeb70724'],
 'www.tianyancha.com/search?key=Matternet': ['brand/b4076153734'],
 'www.tianyancha.com/search?key=Shockwave%20Medical%20Inc.': ['brand/b252b155473'],
 'company/33755398': ['brand/b67ad138237'],
 'company/21543213': ['brand/bafab238300'],
 'company/375260821': ['brand/b238910245'],
 'company/8154215': ['brand/b036f202557'],
 'www.tianyancha.com/search?key=Neuron%20Mobility%20Pte%20Ltd': ['brand/b17c9419013'],
 'company/2352454866': ['brand/b2ce8600457'],
 'company/30766807': ['brand/bcbf6200906'],
 'company/3071807314': ['brand/bf72e559246'],
 'company/2347393339': ['brand/bdef3309397'],
 'company/2320578813': ['brand/b7588514167'],
 'company/3102630482': ['brand/be862322476'],
 'company/3050783950': ['brand/b036d408955'],
 'company/1033548506': ['brand/bfebb134662'],
 'company/2944593082': ['brand/bdbe4149286'],
 'company/2344692971': ['brand/b4898149385'],
 'company/49219785': ['brand/beda347002

55056

### organize
### 工商追踪: organize-company
### 公开投资事件: organize-company
### 未公开投资: organize-company

In [8]:
orgs = set()
org_gongshang = []
org_gktouzi = []
org_wgktouzi = []
k = 1

for line in open('../data/raw/organize_all.json'):
    line = eval(line)
    if k:
        print(line)
        print(line.keys())
        k = 0
    org_id = line['id']
    orgs.add(org_id)
    org_gongshang.extend([(org_id, d.get('CompanyID'), 'org_gongshang') for d in line.get('工商追踪', [])])
    org_gktouzi.extend([(org_id, d.get('CompanyID'), 'org_gktouzi') for d in line.get('公开投资事件', [])])
    org_wgktouzi.extend([(org_id, i.get('企业名称ID'), 'org_wgktouzi') for d in line.get('未公开投资', []) for i in d['投资方']])

{'地址': '北京', '简介': '宜中圣哲成立于2009年，注册资本1500万人民币。', '名称': '宜中圣哲', '工商追踪': [{'企业名称': '北京宜中圣哲投资有限公司', 'CompanyID': 'company/7496101', '投资日期': '2020-06-30', '内容': {'公司名称': '国机重型装备集团股份有限公司', 'CompanyID': 'company/3165467050', '文本': '对外投资: 新增一家公司国机重型装备集团股份有限公司'}, '投资比例': '-'}, {'企业名称': '北京宜中圣哲投资有限公司', 'CompanyID': 'company/7496101', '投资日期': '2019-12-10', '内容': {'公司名称': '青岛北策投资管理有限公司', 'CompanyID': 'company/3396877092', '文本': '对外投资: 新增一家公司青岛北策投资管理有限公司'}, '投资比例': '30.00%'}, {'企业名称': '北京宜中圣哲投资有限公司', 'CompanyID': 'company/7496101', '投资日期': '2019-10-16', '内容': {'公司名称': '山东理工篮园信息科技有限公司', 'CompanyID': 'company/3379354502', '文本': '对外投资: 新增一家公司山东理工篮园信息科技有限公司'}, '投资比例': '55.00%'}, {'企业名称': '北京宜中圣哲投资有限公司', 'CompanyID': 'company/7496101', '投资日期': '2019-09-23', '内容': {'公司名称': '北京数聚领航科技有限公司', 'CompanyID': 'company/3373327041', '文本': '对外投资: 新增一家公司北京数聚领航科技有限公司'}, '投资比例': '30.00%'}, {'企业名称': '北京宜中圣哲投资有限公司', 'CompanyID': 'company/7496101', '投资日期': '2019-07-23', '内容': {'公司名称': '一贝（武汉）科技有限公司', 'CompanyID': 'compa

In [9]:
org_graph = [org_gongshang, org_gktouzi, org_wgktouzi]

In [10]:
len(orgs), len(org_gongshang), len(org_gktouzi), len(org_wgktouzi)

(14290, 96205, 204817, 175110)

### 对节点编号

In [12]:
brand_ord = {d: i for i, d in enumerate(brands)}
comp_ord = {d: i for i, d in enumerate(comps)}
org_ord = {d: i for i, d in enumerate(orgs)}

In [13]:
type_set = {'brand', 'company', 'organize'}
type_ord = {'brand': brand_ord, 'company': comp_ord, 'organize': org_ord}

In [14]:
graph_edge = {}

for graph in [brand_graph, comp_graph, org_graph]:
    for edges in graph:
        for u, v, et in edges:
            if v is None:
                continue
            ut, vt = u.split('/')[0], v.split('/')[0]
            # 类型不符合
            if ut not in type_set or vt not in type_set:
                continue
            # 公司/组织/品牌未爬取
            if u not in type_ord[ut] or v not in type_ord[vt]:
                continue
            if (ut, et, vt) in graph_edge:
                graph_edge[(ut, et, vt)].append((type_ord[ut][u], type_ord[vt][v]))
            else:
                graph_edge[(ut, et, vt)] = [(type_ord[ut][u], type_ord[vt][v])]

In [15]:
graph_edge.keys()

dict_keys([('brand', 'brand_belong', 'company'), ('brand', 'brand_rongzi', 'organize'), ('brand', 'brand_jingpin', 'brand'), ('company', 'comp_gudong', 'company'), ('company', 'comp_touzi', 'company'), ('company', 'comp_jingpin', 'company'), ('company', 'comp_jingpin', 'brand'), ('company', 'comp_gongying', 'company'), ('company', 'comp_lsgudong', 'company'), ('organize', 'org_gongshang', 'company'), ('organize', 'org_gktouzi', 'company'), ('organize', 'org_wgktouzi', 'company')])

In [16]:
graph_data = {k: (torch.tensor(v)[:, 0], torch.tensor(v)[:, 1]) for k, v in graph_edge.items()}

In [17]:
g = dgl.heterograph(graph_data)

In [20]:
g.ntypes, g.etypes, g.canonical_etypes

(['brand', 'company', 'organize'],
 ['brand_belong',
  'brand_jingpin',
  'brand_rongzi',
  'comp_gongying',
  'comp_gudong',
  'comp_jingpin',
  'comp_jingpin',
  'comp_lsgudong',
  'comp_touzi',
  'org_gktouzi',
  'org_gongshang',
  'org_wgktouzi'],
 [('brand', 'brand_belong', 'company'),
  ('brand', 'brand_jingpin', 'brand'),
  ('brand', 'brand_rongzi', 'organize'),
  ('company', 'comp_gongying', 'company'),
  ('company', 'comp_gudong', 'company'),
  ('company', 'comp_jingpin', 'brand'),
  ('company', 'comp_jingpin', 'company'),
  ('company', 'comp_lsgudong', 'company'),
  ('company', 'comp_touzi', 'company'),
  ('organize', 'org_gktouzi', 'company'),
  ('organize', 'org_gongshang', 'company'),
  ('organize', 'org_wgktouzi', 'company')])

In [21]:
g

Graph(num_nodes={'brand': 55056, 'company': 92524, 'organize': 14290},
      num_edges={('brand', 'brand_belong', 'company'): 51039, ('brand', 'brand_jingpin', 'brand'): 1147103, ('brand', 'brand_rongzi', 'organize'): 150568, ('company', 'comp_gongying', 'company'): 67486, ('company', 'comp_gudong', 'company'): 70566, ('company', 'comp_jingpin', 'brand'): 275658, ('company', 'comp_jingpin', 'company'): 252377, ('company', 'comp_lsgudong', 'company'): 15120, ('company', 'comp_touzi', 'company'): 70047, ('organize', 'org_gktouzi', 'company'): 135706, ('organize', 'org_gongshang', 'company'): 49446, ('organize', 'org_wgktouzi', 'company'): 103839},
      metagraph=[('brand', 'company', 'brand_belong'), ('brand', 'brand', 'brand_jingpin'), ('brand', 'organize', 'brand_rongzi'), ('company', 'company', 'comp_gongying'), ('company', 'company', 'comp_gudong'), ('company', 'company', 'comp_jingpin'), ('company', 'company', 'comp_lsgudong'), ('company', 'company', 'comp_touzi'), ('company', 'bra

In [28]:
torch.save([g, type_ord], path + 'heter_graph.pt')

In [37]:
g.num_nodes(), g.num_edges()

(161870, 2388955)

In [38]:
(51039+1147103+150568)/55056

24.49705754141238

In [39]:
1147103/55056, 51039/55056, 150568/55056

(20.835204155768672, 0.9270379250217959, 2.7348154606219124)

In [41]:
[(67486+70566+252377+15120+70047)/92524, 275658/92524]

[5.140244693268773, 2.979313475422593]

In [42]:
[(135706+49446+103839)/14290]

[20.223303009097272]

In [45]:
f = open(path + 'brand_all.json')
brand_labels = []
brand_members = []
for line in f:
    line = eval(line)
    brand_labels.extend(line['标签'])
    if '核心成员' in line:
        brand_members.extend(line['核心成员'])

In [46]:
from collections import Counter

In [57]:
sorted(dict(Counter(brand_labels)).items(), key=lambda d: d[1], reverse=True)

[('生产制造', 11354),
 ('企业服务', 10416),
 ('医疗健康', 6198),
 ('硬件', 5699),
 ('文娱传媒', 5691),
 ('企业通用服务', 5167),
 ('金融', 4776),
 ('制造业', 3818),
 ('电子商务', 3786),
 ('人工智能', 3573),
 ('移动互联网', 3412),
 ('汽车交通', 3075),
 ('先进制造', 2914),
 ('团队优秀', 2783),
 ('仪器设备', 2761),
 ('垂直领域硬件', 2718),
 ('A股产业图谱', 2576),
 ('信息技术', 2426),
 ('材料类', 2397),
 ('教育培训', 2365),
 ('医药服务', 2252),
 ('物联网', 2215),
 ('游戏', 2184),
 ('大数据', 2114),
 ('生活服务', 2079),
 ('工业', 2017),
 ('房产家居', 1914),
 ('专用设备', 1906),
 ('电子设备', 1897),
 ('电商经营模式', 1868),
 ('人工智能应用场景', 1819),
 ('数据服务', 1740),
 ('垂直电商', 1737),
 ('游戏开发', 1719),
 ('B2C', 1682),
 ('医疗器械', 1575),
 ('投资理财', 1564),
 ('物联网应用层', 1545),
 ('通用设备', 1500),
 ('推广营销', 1498),
 ('O2O', 1490),
 ('行业解决方案', 1470),
 ('企业管理软件', 1410),
 ('媒体', 1389),
 ('游戏研发发行', 1388),
 ('企业IT服务', 1351),
 ('建筑', 1341),
 ('智能硬件', 1340),
 ('关键元器件', 1340),
 ('机械设备', 1289),
 ('软件与服务', 1286),
 ('能源电力', 1281),
 ('清华系', 1263),
 ('资本货物', 1260),
 ('新基建', 1258),
 ('消费升级', 1247),
 ('电气机械和器材', 1242),
 ('农业', 1233),
 ('硬科技

In [58]:
len(dict(Counter(brand_labels)))

35387

In [61]:
len(brand_members)

185944

In [62]:
185944/55056

3.377361232199942

In [71]:
f = open(path + 'company_total.json')
eval(next(f))['企业业务']

[{'产品名称': '优课达',
  'BrandID': 'brand/b43c0469752',
  '所属地': '浙江',
  '标签': '教育培训',
  '日期': '2017-09-12'}]

In [ ]:
"仅20年，互联网就完成了“传统IT行业—互联网—移动互联网—人工智能—IOT”的更迭，技术直接对世界带来影响所用的时间从10年缩至5年，且速度越来越快。优课达团队希望通过累积的经验，为热爱编程的同学们规划科学的学习路径，更快更好地学写代码；通过各种测试让同学们对自己有更加清晰的认知； 通过简历指导和大厂内推，让每个优秀的人都有闪光的机会！".count

In [73]:
f = open(path + 'company_total.json')
comp_label = []

for line in f:
    line = eval(line)
    if '企业业务' in line:
        comp_label.extend([d['标签'] for d in line['企业业务'] if '标签' in d])

In [75]:
len(Counter(comp_label))

41

In [76]:
sorted(dict(Counter(comp_label)).items(), key=lambda d: d[1], reverse=True)

[('企业服务', 6445),
 ('生产制造', 6367),
 ('医疗健康', 4638),
 ('金融', 3511),
 ('文娱传媒', 2813),
 ('电子商务', 2392),
 ('汽车交通', 2121),
 ('教育培训', 2012),
 ('游戏', 1643),
 ('硬件', 1286),
 ('房产家居', 1105),
 ('先进制造', 1084),
 ('生活服务', 970),
 ('社交社区', 916),
 ('人工智能', 868),
 ('旅游户外', 809),
 ('工具软件', 804),
 ('建筑', 793),
 ('能源电力', 731),
 ('食品饮料', 714),
 ('农业', 711),
 ('物流运输', 683),
 ('环保', 612),
 ('餐饮业', 526),
 ('体育健身', 489),
 ('批发零售', 438),
 ('大数据', 434),
 ('物联网', 426),
 ('VR/AR', 392),
 ('区块链', 321),
 ('服装纺织', 308),
 ('开采', 230),
 ('科研及技术服务', 182),
 ('政务及公共服务', 56),
 ('消费升级', 51),
 ('硬科技', 29),
 ('-', 7),
 ('新基建', 6),
 ('文化娱乐', 1),
 ('其他', 1),
 ('服务业', 1)]

In [82]:
comp_members = [i for i in list(zip(*comp_gudong))[1] if i is not None and 'human' in i]

In [84]:
len(comp_members)

210000

In [85]:
len([i for i in list(zip(*comp_lsgudong))[1] if i is not None and 'human' in i])

93789

In [86]:
(210000+93789)/92524

3.283353508278933

In [87]:
edges = {('brand', 'brand_belong', 'company'): 51039, ('brand', 'brand_jingpin', 'brand'): 1147103, ('brand', 'brand_rongzi', 'organize'): 150568, ('company', 'comp_gongying', 'company'): 67486, ('company', 'comp_gudong', 'company'): 70566, ('company', 'comp_jingpin', 'brand'): 275658, ('company', 'comp_jingpin', 'company'): 252377, ('company', 'comp_lsgudong', 'company'): 15120, ('company', 'comp_touzi', 'company'): 70047, ('organize', 'org_gktouzi', 'company'): 135706, ('organize', 'org_gongshang', 'company'): 49446, ('organize', 'org_wgktouzi', 'company'): 103839}

In [88]:
sorted(dict(Counter(edges)).items(), key=lambda d: d[1], reverse=True)

[(('brand', 'brand_jingpin', 'brand'), 1147103),
 (('company', 'comp_jingpin', 'brand'), 275658),
 (('company', 'comp_jingpin', 'company'), 252377),
 (('brand', 'brand_rongzi', 'organize'), 150568),
 (('organize', 'org_gktouzi', 'company'), 135706),
 (('organize', 'org_wgktouzi', 'company'), 103839),
 (('company', 'comp_gudong', 'company'), 70566),
 (('company', 'comp_touzi', 'company'), 70047),
 (('company', 'comp_gongying', 'company'), 67486),
 (('brand', 'brand_belong', 'company'), 51039),
 (('organize', 'org_gongshang', 'company'), 49446),
 (('company', 'comp_lsgudong', 'company'), 15120)]

In [ ]:
-